# Baseline Model Experiment 

This notebook contains a baseline model experiment with a simple Artificial Neural Network (ANN) using [FastAI Tabular Learner](https://docs.fast.ai/tabular.learner.html). We begin experiments with this simple model on the premise that - when taking into account the level of effort - clever feature engineering typically outperforms employing alternative model types (such as using LSTMs). 

A few items about the experiment: 

- We train for `D` days (periods) and predict on the next day in order to train the model. After each complete period, the model is retrained on the new data (either from scratch or with transfter learning). We model the experiment this way to coincide with how a model would be used in practice and to account for the fast that the most recent data has an outsized influence on model performance. 
- We identify and utilize periods of observations for model training and testing that are greater than some threshold `period_size`. Deep neural networks cannot be trained with missing data. 
- Related to the above, it would be an inappropriate training strategy to simply concatenate the periods together during the training process - the value(s) at the end of a period does not help predict the values at the beginning of the next period. As such, we will need to retrain the model for each subsequent day in the training set using transfer learning. The same testing set is used throughout the model training process. 
- An appropriate learning rate is determined using an automated learning rate finder. 
- The model is trained using an early stopping criterion that is intened to ovoid model overfitting. 
- A selection of possible model architectures for the ANN (nodes in the layers, drop out rates, etc). is available for the experiments. 

## Imports 

In [1]:
import sys; sys.executable

'/home/vconstan/.conda/envs/tsunami/bin/python'

In [2]:
from fastai.tabular import *
from fastai.metrics import *
from fastai import torch_core
from fastai.callbacks import *
from fastai.callbacks.mem import PeakMemMetric
from fastai.utils.mod_display import *

import pandas as pd
from pathlib import Path
import plotly.graph_objects as go
from PyNomaly import loop
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
import seaborn as sns
from scipy import spatial, stats
from src import data
import time
import torch
from tqdm.notebook import tqdm

## Experiment Parameters

First, we can specify and select a variety of architectures for the model. More specifically, the below architectures are used by FastAI's tabular learner. 

In [3]:
architectures = {
    1: {
        "layers": [10, 500, 1000, 2500, 25000, 2500, 1000, 500, 10],
        "ps": [0.0, 0.1, 0.2, 0.2, 0.25, 0.2, 0.2, 0.1, 0.0]
    },
    2: {
        "layers": [10, 50, 100, 250, 1000, 250, 100, 50, 10],
        "ps": [0.5, 0.4, 0.3, 0.2, 0.2, 0.1, 0.05, 0.025, 0.0]
    },
    3: {
        "layers": [10, 50, 100, 250, 1000, 250, 100, 50, 10],
        "ps": [0.0, 0.025, 0.05, 0.1, 0.2, 0.2, 0.3, 0.4, 0.5]
    },
    4: {
        "layers": [50000, 5000, 1000, 500, 25, 1],
        "ps": [0.2, 0.15, 0.1, 0.05, 0.025, 0.]
    },
    5: {
        "layers": [50000, 7500, 2500, 1000, 250, 80, 25, 1],
        "ps": [0.25, 0.2, 0.15, 0.1, 0.05, 0.025, 0.01, 0]
    },
    6: {
        "layers": [1000, 250, 50, 1],
        "ps": [0.2, 0.1, 0.025, 0.]
    },
    7: {
        "layers": [3, 3],
        "ps": [0.1, 0.0]
    },
    8: {
        "layers": [10, 10, 10, 10, 10, 10],
        "ps": [0.1, 0.1, 0.1, 0.1, 0.05, 0.0]
    }
}

Other experimental parameters are set below, such as which data to use in the experiment, which satellite to model, and other model specifications. A single model is applied to each satellite, so the below would be repeated over more than one satellite to conduct a thorough experiment. 

In [4]:
from typing import Union

In [5]:
class Experiment(object): 
    
    
    def __init__(self, 
                 dependent_variable: str, 
                 independent_variables: list, 
                 satellites: list, 
                 model: tabular_learner, 
                 year: int = 2012, 
                 location: str = "hawaii", 
                 elevation_filter: Union[int, None] = None, 
                 time_aggregation: str = "1min", 
                 batch_normalization: bool = True, weight_decay: float = 0.1, 
                 model_save_directory: str = "", model_name: str = "model-latest",
                 max_epochs: int = 500, cuda_device: int = 0):
        
        # experiment setup
        self.year = year
        self.location = location
        # TODO: below path will need rework when placed in src/modeling.py
        self.data_paths = Path('../data/' + LOCATION + '/' + str(YEAR))
        self.days = [str(f).split("/")[-1] for f in self.data_paths.iterdir() if f.is_dir()]
        self.satellites = satellites
        self.elevation_filter = elevation_filter
        self.time_aggregation = time_aggregation, 
        self.batch_normalization = batch_normalization
        self.weight_decay = weight_decay
        self.model_save_directory = model_save_directory
        self.model_name = model_name
        self.max_epochs = max_epochs
        self.cuda_device = cuda_device
        self.model = model
        # TODO: important note that the data as needed for the model should be passed 
        self.data = data
        
        # TODO: experiment results 
        
    def set_data(self) -> None: 
        
        # read all of the data into a dictionary 
        dataframes = dict()
        for d in DAYS:
            print("\n--- " + str(d) + "---")

            # read in the data 
            df = data.read_day(
                location=LOCATION,
                year=YEAR,
                day_of_year=int(d)
            )
            dataframes[d] = df
        
        
    

    
    

## Read Data

This experiment uses data from the Hawaii dataset. We will train the model on a period of `D` days (periods) and test on the latest day (period). The day of the earthquake will represent the _validation_ set, data that is unseen during the model training process. 

In [6]:
dataframes = dict()
for d in DAYS:
    print("\n--- " + str(d) + "---")
    
    # read in the data 
    df = data.read_day(
        location=LOCATION,
        year=YEAR,
        day_of_year=int(d)
    )
    dataframes[d] = df

NameError: name 'DAYS' is not defined

In [ ]:
# concatenate the dataframes loaded previously into one large dataframe 
df_all = pd.concat([dataframes[d] for d in dataframes.keys()]) 

## Prep Dataset

The above experimental parameter definitions will be interesting to explore. However, we also need to be smart about which data we feed into the model for training and how to feet that data into the model as well in order to set a good baseline for our experiments. 

The data contains missing values and - in practice and in the real-world - and we'll need to account for that in our modeling strategy. We'll want to train the model on each chunk of data to come in, and a chunk represents a continuous stretch of data without any missing values. 

In [ ]:
df_sat = df_all.filter(regex=GROUND_STATION + "__" + SAT, axis=1)

In [ ]:
df_sat.head()

In [7]:
df_sat.shape

NameError: name 'df_sat' is not defined

### Elevation Filter 

Exclude any observations recorded below a specific elevation specified. 

In [ ]:
if ELE_FILTER is not None: 
    df_sat = df_sat[df_sat[GROUND_STATION + "__" + SAT + "_ele"] > ELE_FILTER]

In [ ]:
df_sat.shape

### Resample

In [ ]:
df_model = df_sat.dropna().resample(TIME_AGG).mean()

### Remove Any Stray Outliers 

While the elevation filtering does help to provide clean, consistent data to the model, it is a possibility that the connection between the ground station and the satellite can sometimes cause inconsistent / noisy TEC estimates. These are not indicative of normal behavior, so we will want to remove them from our dataset. 

This behavior **only occurs for the G04 satellite**, so we will want to 

We can use a density-based local outlier approach to identify and later remove these stray values from our training set. This should improve our results and simply the model training task to some degree.

In [ ]:
filter_index = df_sat.index.to_series().between('2012-10-22', '2012-10-23 23:59:00')

In [ ]:
df_sat_outliers = df_sat[filter_index]

In [ ]:
m = loop.LocalOutlierProbability(
    df_sat_outliers[
        [
            GROUND_STATION + "__" + SAT
        ]
    ], 
    extent=3, 
    n_neighbors=500
).fit()
scores = m.local_outlier_probabilities

In [ ]:
scores

In [ ]:
df_sat_outliers["outlier_scores"] = scores

In [ ]:
df_sat_outliers

In [ ]:
# create subplots similar to the paper
values = list()
for val in df_sat_outliers:
    if val != 'outlier_scores':
        values.append(val)


sns.set(style="darkgrid")
f, axes = plt.subplots(len(values), 1, figsize=(25,25), sharex=True)
i = 0

for val in values:
    ax = sns.lineplot(x=df_sat_outliers.index, y=val, ax=axes[i], data=df_sat_outliers, color="gray")
    ax.lines[0].set_linestyle("--")
    
    ax2 = sns.scatterplot(x=df_sat_outliers.index, y=val,
                data=df_sat_outliers, ax=axes[i],
                hue="outlier_scores", 
                palette="bwr",
                legend=False)
    
    i += 1


plt.show()

The red dots above indicate points which may be considered as more likely to be anomalous by the Local Outlier Probability (LoOP) outlier detection approach. Our strategy will be to remove any values from the data that are close to these detected points. 

In [ ]:
REMOVAL_WINDOW = 10 # number of minutes / points before and after 
OUTLIER_THRESHOLD = 0.9 # on a scale [0, 1]

In [ ]:
# for each value in df_sat_outliers that matches the threshold
# identify the idx and do the thing 

In [ ]:
outliers = df_sat_outliers[df_sat_outliers["outlier_scores"] > OUTLIER_THRESHOLD]
outliers

In [ ]:
removal_windows = list()
list_index = list(df_sat_outliers.index.values)
for idx in outliers.index.values:
    idx_i = list_index.index(idx)
    idx_range = [idx_i - REMOVAL_WINDOW, idx_i + REMOVAL_WINDOW]
    removal_windows.append(idx_range)
for rw in removal_windows:
    df_sat_outliers.iloc[rw[0]:rw[1], :] = None    

In [ ]:
# create subplots similar to the paper
values = list()
for val in df_sat_outliers:
    if val != 'outlier_scores':
        values.append(val)

sns.set(style="darkgrid")
f, axes = plt.subplots(len(values), 1, figsize=(25,25), sharex=True)
i = 0

for val in values:
    ax = sns.lineplot(x=df_sat_outliers.index, y=val, ax=axes[i], data=df_sat_outliers, color="gray")
    ax.lines[0].set_linestyle("--")
    
    ax2 = sns.scatterplot(x=df_sat_outliers.index, y=val,
                data=df_sat_outliers, ax=axes[i],
                color="blue",
                legend=False)
    
    i += 1


plt.show()

Now that we are comfortable with our outlier removal approach, let's apply it to the modeling dataset. 

In [ ]:
removal_windows = list()
list_index = list(df_sat.index.values)
for idx in outliers.index.values:
    idx_i = list_index.index(idx)
    idx_range = [idx_i - REMOVAL_WINDOW, idx_i + REMOVAL_WINDOW]
    removal_windows.append(idx_range)
for rw in removal_windows:
    df_sat.iloc[rw[0]:rw[1], :] = None    

In [ ]:
df_model.shape

In [ ]:
# data 
YEAR = 2012 # can be 2012, 2015
LOCATION = "hawaii" # can be hawaii, chile
DATA_PATHS = Path('../data/' + LOCATION + '/' + str(YEAR))
DAYS = [str(f).split("/")[-1] for f in DATA_PATHS.iterdir() if f.is_dir()]
GROUND_STATION = "ahup" # string
SAT = "G07" # string
ELE_FILTER = 35 # int or None
TIME_AGG = "1Min"
BATCH_NORM = True
WEIGHT_DECAY = 0.1
USE_MISSING_INDICATOR = False # create features that inform the model of missing data 

# model specification 
MODEL_SAVE_DIR = ""
MODEL_NAME = "model-latest"
if MODEL_SAVE_DIR == "":
    MODEL_LOCATION = MODEL_NAME
else:
    MODEL_LOCATION = MODEL_SAVE_DIR + "/" + MODEL_NAME
MODEL_ARCHITECTURE = 7
BATCH_SIZE = 16
DEP = GROUND_STATION + "__" + SAT
FEATURES = [
#     GROUND_STATION + "__" + SAT,
    GROUND_STATION + "__" + SAT + "_ele",
#     GROUND_STATION + "__" + SAT + "_lat",
#     GROUND_STATION + "__" + SAT + "_lon",
#     GROUND_STATION + "__" + SAT + "_h_ipp"
]
MAX_EPOCHS = 500

## Select a CUDA Device

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

In [ ]:
DEVICE = 3

In [ ]:
torch.cuda.set_device(DEVICE)

In [ ]:
cuda_device = torch.device('cuda:' + str(DEVICE))

In [ ]:
torch.cuda.current_device()

One useful command for monitoring GPU utilization is one from `nvidia-smi`:

```bash
nvidia-smi -q -g 0 -d UTILIZATION -l
```

Or: 

```bash
gpustat -cp -i 1
```

### Add a Missing Indicator

In [ ]:
df_model["missing"] = df_model.apply(
    lambda row: [1 if any(row.isna()) else 0][0],
    axis=1
)

In [ ]:
df_model["missing"].value_counts()

In [ ]:
df_model[df_model["missing"] == 1].head()

In [ ]:
df_model[df_model["missing"] == 0].head()

### Visualize Available Data

#### All Base Features for The Satellite and Ground Station

In [ ]:
# create subplots that show the values in the data
values = list()
for val in df_sat:
    values.append(val)


sns.set(style="darkgrid")
f, axes = plt.subplots(len(values), 1, figsize=(100,25), sharex=True)
i = 0

for val in values:
    ax = sns.lineplot(x=df_sat.index, y=val, ax=axes[i], data=df_sat, color="gray")
    ax.lines[0].set_linestyle("--")
    
    ax2 = sns.scatterplot(x=df_sat.index, y=val,
                data=df_sat, ax=axes[i],
                color="blue")
    
    i += 1


plt.show()

#### For the Day of the Earthquake 

In [ ]:
# get those dates from the 28th of the month, day of the earthquake
filter_index = df_sat.index.to_series().between('2012-10-28', '2012-10-28 23:59:00')

In [ ]:
df_sat_earthquake = df_sat[filter_index]

In [ ]:
df_sat_earthquake

In [ ]:
# create subplots similar to the paper
values = list()
for val in df_sat_earthquake:
    values.append(val)


sns.set(style="darkgrid")
f, axes = plt.subplots(len(values), 1, figsize=(25,25), sharex=True)
i = 0

for val in values:
    ax = sns.lineplot(x=df_sat_earthquake.index, y=val, ax=axes[i], data=df_sat_earthquake, color="gray")
    ax.lines[0].set_linestyle("--")
    
    ax2 = sns.scatterplot(x=df_sat_earthquake.index, y=val,
                data=df_sat_earthquake, ax=axes[i],
                color="blue")
    
    i += 1


plt.show()

### Split The Data Into Periods 

These periods are defined by consecutive empty (NaN) values in the dataframe. The data is only available for the satellite as it passes close to the ground station on each day. We will train the data in a similar way, ensuring our approach is compatible with the constraints in the operating environment. 

In [ ]:
def split_dataframe(dataframe: pd.DataFrame) -> list: 

    # handle missing values and "chunk" the data for training and testing 
    events = np.split(dataframe, np.where(np.isnan(dataframe))[0])

    # removing NaN entries
    events = [ev[~np.isnan(ev)] for ev in events if not isinstance(ev, np.ndarray)]

    # removing empty DataFrames
    events = [ev.dropna() for ev in events if not ev.empty and ev.shape[0] > 100]

    return events

In [ ]:
events = split_dataframe(df_model)
len(events)

In [ ]:
events[1]

We know from domain knowledge that each period in the data corresponds to a specific day. The third day in the dataset (`302`) corresponds to the third period (index `2`) in the dataset. 

All of the experiments will attempt at detecting the anomaly using an analysis of the residual values using the day of the earthquake, which will be contained in the `validation` set. Data prior to the day of the earthquake, with the day prior to the earthquake being used for `test` data. Any days prior to that are considered as training data. 

At some point in the future, we will want to setup a controlled trail of how this would operate in practice (daily retraining of models). For now, we will focus on the task of training the model for the first time. 

### Scale the Data 

Deep learning models do not perform well when model inputs are not scaled appropriately. In this application, we will scale the data for each feature to a scale of -1 to 1, and do so separetely for each day. This is again driven by operational considerations. 

In [ ]:
normalized_events = list()
for ev in events: 
    
    # for each column in the data, rescale -1 to 1 
    col_data = list()
    for col in ev.columns.values:
        
        normalized_data = minmax_scale(
                    ev[col].dropna(), 
                    feature_range=(-1, 1)
                )
        col_data.append(normalized_data)
        
    df_period = pd.DataFrame(np.array(col_data).T, columns=list(ev.columns.values) )
    df_period["timestamp"] = ev[col].index
    df_period.index = df_period["timestamp"]
    df_period = df_period.drop(columns=["timestamp"])
    
    # convert to seconds of the day for later annotation 
    df_period["sod"] = (df_period.index.hour*60+df_period.index.minute)*60 + df_period.index.second
    
    if USE_MISSING_INDICATOR:
        df_period["missing"] = ev["missing"]
    
    normalized_events.append(df_period)


In [ ]:
len(normalized_events)

In [ ]:
normalized_events[13].shape

In [ ]:
normalized_events[12]

### Allocate Data for Modeling

In [ ]:
def allocate_periods(periods: list, index_earthquake: int) -> dict:
    
    data = dict()
    
    data["valid"] = periods[index_earthquake]
    data["test"] = periods[index_earthquake - 1]
    data["train"] = periods[0:index_earthquake - 1]
    
    return data

In [ ]:
df_model_by_period = allocate_periods(normalized_events, 12)

## Train the Model 

In [ ]:
print(len(df_model_by_period["train"]))

The below function preps the data for modeling in Fast AI. 

In [ ]:
## Work in progress
def make_dataBunch(df_train: pd.DataFrame, df_test: pd.DataFrame, df_valid: pd.DataFrame, features: list, dependent: str, include_catvars: bool = False, catvars: list = [], batch_size: int = 256):
    """
    Creates a TabularDataBunch to feed as an input 
    into the learner. 
    """
    
    valid_start_index = df_train.shape[0] + 1
    valid_end_index = df_train.shape[0] + df_test.shape[0]
    
    df_train_validation = pd.concat([df_train, df_test])
        
    # create the data bunch
    if include_catvars:
        data = TabularDataBunch.from_df(
            "models", 
            df_train_validation[features + [dependent]], 
            dependent, 
            valid_idx=np.array(list(range(valid_start_index, valid_end_index))),
#             test_df=df_test, 
            procs=[Categorify],
            bs=batch_size, # batch size
            cat_names=catvars,
            device=cuda_device,
            num_workers=0
        )
    else:
        data = TabularDataBunch.from_df(
            "models", 
            df_train_validation[features + [dependent]], 
            dependent, 
            valid_idx=np.array(list(range(valid_start_index, valid_end_index))),
#             test_df=df_test, 
            procs=None, # disable any automatic preprocessing
            bs=batch_size, # batch size
            device=cuda_device,
            num_workers=0
        )
 
    return {
        "databunch": data, 
        "train": df_train, 
        "test": df_test, 
        "valid": df_valid
    }

### Determine the Learning Rate

We utilize an [automatic learning rate finder](https://forums.fast.ai/t/automated-learning-rate-suggester/44199/8) to determine the ideal learning rate automatically. While this approach does not always guarantee that the perfect learning rate is found, in practice we have found the approach to work well and has been quite stable.

In [ ]:
def find_appropriate_lr(model:Learner, lr_diff:int = 15, loss_threshold:float = .05, adjust_value:float = 1, plot:bool = False) -> float:
    #Run the Learning Rate Finder
    model.lr_find(
        end_lr=2.,
        stop_div=False # continues through all LRs as opposed to auto stopping
    )
    
    #Get loss values and their corresponding gradients, and get lr values
    losses = np.array(model.recorder.losses)
    assert(lr_diff < len(losses))
    loss_grad = np.gradient(losses)
    lrs = model.recorder.lrs
    
    #Search for index in gradients where loss is lowest before the loss spike
    #Initialize right and left idx using the lr_diff as a spacing unit
    #Set the local min lr as -1 to signify if threshold is too low
    r_idx = -1
    l_idx = r_idx - lr_diff
    while (l_idx >= -len(losses)) and (abs(loss_grad[r_idx] - loss_grad[l_idx]) > loss_threshold):
        local_min_lr = lrs[l_idx]
        r_idx -= 1
        l_idx -= 1

    lr_to_use = local_min_lr * adjust_value
    
    if plot:
        # plots the gradients of the losses in respect to the learning rate change
        plt.plot(loss_grad)
        plt.plot(len(losses)+l_idx, loss_grad[l_idx],markersize=10,marker='o',color='red')
        plt.ylabel("Loss")
        plt.xlabel("Index of LRs")
        plt.show()

        plt.plot(np.log10(lrs), losses)
        plt.ylabel("Loss")
        plt.xlabel("Log 10 Transform of Learning Rate")
        loss_coord = np.interp(np.log10(lr_to_use), np.log10(lrs), losses)
        plt.plot(np.log10(lr_to_use), loss_coord, markersize=10,marker='o',color='red')
        plt.show()
        
    return lr_to_use

We will train the model for each day in the training set. 

### Time Series Cross Validation

Time-Series CV results will be tracked. Will contain the following: 

- `root_mean_square_error`
- `period`
- `learn_rate` 
- `training_time_seconds`

In [ ]:
time_series_cv_log = list()
cv_log_cols = ["root_mean_square_error", "period", "learn_rate", "training_time_seconds"]

# learning_rates = [
#     0.001,
#     0.0005,
#     0.0001,
#     0.00005,
#     0.00001,
#     0.000005,
#     0.000001,
#     0.0000005,
#     0.0000001,
#     0.00000005,
#     0.00000001
# ]


lr = None
# for i in range(0, len(df_model_by_period["train"])):
    
#     print("\n---------- PERIOD: " + str(i) + " ----------")
    
# get the current time 
now = time.time()

#     # TODO: TEST THE BELOW
#     # if this isn't the first time a model is trained, load it from memory 
#     if i != 0:
#         # load learner
#         lr = lr.load('model-latest')  

# first, create a data bunch for this round of the modeling process        

# below line "hacked" to use dataset as one large one 
#     df_train = df_model_by_period["train"][i]



# df_train = pd.concat(df_model_by_period["train"])
if USE_MISSING_INDICATOR is True:
    df_train = pd.concat(df_model_by_period["train"]).resample("1Min").mean()
    # fill with mean everywhere everywhere 
    for col in df_train.columns.values:
                
        df_train[col] = df_train[col].fillna(np.mean(df_train[col].dropna().values))

    FEATURES += ["missing"]
    
else:
    df_train = pd.concat(df_model_by_period["train"])
        
# print(df_train.sample(frac=1).head(10))

df_train[DEP + "_target"] = df_train[DEP]
df_test = df_model_by_period["test"]
df_test[DEP + "_target"] = df_test[DEP]
df_valid = df_model_by_period["valid"]
df_valid[DEP + "_target"] = df_valid[DEP]

data_bunch = make_dataBunch(
    df_train, 
    df_test,
    df_valid,
    FEATURES, 
    DEP + "_target", 
    include_catvars=False, 
    catvars=None, 
    batch_size=BATCH_SIZE
)

# define the model 
if lr is None: 
    lr = tabular_learner(
        data_bunch["databunch"], 
        layers=architectures[MODEL_ARCHITECTURE]["layers"], 
        ps=architectures[MODEL_ARCHITECTURE]["ps"],
        metrics=[root_mean_squared_error], 
        callback_fns=[CSVLogger, PeakMemMetric],
        use_bn=BATCH_NORM,
        wd=WEIGHT_DECAY
    )

#     # automatically find the ideal learning rate 
#     try:
#         learn_rate = find_appropriate_lr(
#             model=lr,
#             plot=True
#         )

#     except:
#         learn_rate = 0.001
#         print("ERROR: cannot determine learning rate.")

# learn_rate = learning_rates[i]
learn_rate = 0.0001

print("Learning rate: " + str(learn_rate))

# train the model 
lr.fit_one_cycle(
    MAX_EPOCHS, # max epochs
    learn_rate,
    callbacks=[
        SaveModelCallback(
            lr, 
            every='epoch', 
            monitor=['accuracy', 'root_mean_square_error']
        ),
        EarlyStoppingCallback(
            lr,
            monitor='valid_loss', #'valid_loss', 'root_mean_square_error'
            min_delta=0.0001, # 0.0001
            patience=15
        ),
        ShowGraph(
            lr
        )
    ]
)

# loaded learners do not have a recorder
lr.recorder.plot_losses()
lr.recorder.plot_metrics()

# last set of scores for this training cycle 
rmse = lr.recorder.metrics[-1][0].item()

# get the end time 
later = time.time()

# get time time difference in seconds
time_diff = int(later - now)

time_series_cv_log.append(
#     [rmse, i, learn_rate, time_diff]
    [rmse, 0, learn_rate, time_diff]
)

# save learner and export the model weights
lr.save(MODEL_LOCATION)
lr.export(MODEL_LOCATION + '-export.pkl')
    
#     # destroy the learner and clear the mem cache 
#     lr.destroy()
#     torch.cuda.empty_cache() 
        
print("Results from last training set.")
print(lr.show_results())
   
time_series_cv_results_df = pd.DataFrame(
    time_series_cv_log,
    columns=cv_log_cols
)               

In [ ]:
# destroy the learner and clear the mem cache
# lr.destroy()
# torch.cuda.empty_cache() 

In [ ]:
time_series_cv_results_df
# TODO: add num epochs trained 
# TODO: add total training time for period 

Note that the above training process is specific to a satellite and ground station. In order to report many of the metrics below, we will need to (at a later date) perform the training and testing process over multiple satellite and ground station combinations. 

Now that the model training is complete, load the learner from disk. 

In [ ]:
# lr = tabular_learner(
#         data_bunch["databunch"], 
#         layers=architectures[MODEL_ARCHITECTURE]["layers"], 
#         ps=architectures[MODEL_ARCHITECTURE]["ps"],
#         metrics=[root_mean_squared_error], 
#         callback_fns=[CSVLogger, PeakMemMetric],
#         use_bn=BATCH_NORM,
#         wd=WEIGHT_DECAY
#     )

In [ ]:
# # load learner
# lr = lr.load('model-latest') 

In [ ]:
# show me a summary look at the results 
lr.show_results()

Plot the values on the testing set used for model training. 

In [ ]:
# function to predict values using input data from new data

def predict_values(dataframe, learner, dependent, frac=1.0):
    
    """
    Using the passed learner, predicts the appropriate value given the input data 
    and generates errors for analysis. 
    """
    
    # get a sample of the dataset 
    dataframe_pred = dataframe.copy()
    n_obs = int(dataframe_pred.shape[0] * frac)
    idx = random.sample(range(0, dataframe_pred.shape[0]), n_obs)
    dataframe_pred = dataframe_pred.iloc[idx, :]
    
    # get the predictions
    
    # TODO: update this to work with gpu over large samples like the other function 
    predictions = []
    print('Generating predictions and errors...')
    for idx, row in tqdm(dataframe_pred.iterrows(), total=dataframe_pred.shape[0]):        
        predictions.append(learner.predict(row)[1].numpy()[0])
    
    
    dataframe_pred["predicted"] = predictions
    dataframe_pred['error'] = dataframe_pred['predicted'] - dataframe_pred[dependent]
    dataframe_pred['absolute_error'] = np.abs(dataframe_pred['error'])
    dataframe_pred["timestamp"] = dataframe_pred.index
    
    return dataframe_pred
    

Generate a dataframe from the predicted values on the `test` set. 

In [ ]:
df_test = predict_values(
    dataframe=data_bunch["test"], # TODO: currently using latest databunch from prev loop
    learner=lr, # TODO: currently using latest learner from prev loop
    dependent=DEP + "_target",
    frac=1.
).sort_index()

Plot the predicted (blue) versus the actual (gray) values. 

In [ ]:
plt.figure(figsize=(30, 10))

ax = sns.lineplot(x=df_test.index, y=DEP + "_target", data=df_test, color="gray")
ax.lines[0].set_linestyle("--")

ax2 = sns.scatterplot(x=df_test.index, y=DEP + "_target",
            data=df_test,
            color="gray")


ax3 = sns.lineplot(x=df_test.index, y="predicted", data=df_test, color="blue")

plt.show()

Plot the absolute error. 

In [ ]:
plt.figure(figsize=(30, 10))


ax = sns.lineplot(x=df_test.index, y="absolute_error", data=df_test, color="red")
# ax.lines[0].set_linestyle("--")

# ax2 = sns.scatterplot(x=df_assess_day.index, y="absolute_error",
#             data=df_assess_day,
#             color="darkgray")

plt.show()

In [ ]:
# simple moving average 
df_test["absolute_error_sma_3"] = df_test["absolute_error"].rolling(3, min_periods=1).mean()
df_test["absolute_error_sma_5"] = df_test["absolute_error"].rolling(5, min_periods=1).mean()
df_test["absolute_error_sma_10"] = df_test["absolute_error"].rolling(10, min_periods=1).mean()

In [ ]:
plt.figure(figsize=(30, 10))


ax = sns.lineplot(x=df_test.index, y="absolute_error", data=df_test, color="gray")
ax = sns.lineplot(x=df_test.index, y="absolute_error_sma_3", data=df_test, color="red")
# ax.lines[0].set_linestyle("--")

# ax2 = sns.scatterplot(x=df_assess_day.index, y="absolute_error",
#             data=df_assess_day,
#             color="darkgray")

plt.show()

## Identify and Classify Tsunami-Related Disturbances

Using the validation data - the day of the earthquake - use the residuals to detect the anomaly. 

This section is a **work in progress**. In order to estimate the model's `accuracy`, `recall`, `precision`, `F-score`, and `coverage`, we need to be able to classify specific time periods as anomalous - in this context, as ionoshperic disturbances related to tsunami waves. 

In [ ]:
df_assess = predict_values(
    dataframe=data_bunch["valid"], # TODO: currently using latest databunch from prev loop
    learner=lr, # TODO: currently using latest learner from prev loop
    dependent=DEP + "_target",
    frac=1.
).sort_index()

In [ ]:
df_assess.sample(frac=1).head()

### Plot Actual and Predicted Values

In [ ]:
plt.figure(figsize=(30, 10))

ax = sns.lineplot(x=df_assess.index, y=DEP + "_target", data=df_assess, color="gray")
ax.lines[0].set_linestyle("--")

ax2 = sns.scatterplot(x=df_assess.index, y=DEP + "_target",
            data=df_assess,
            color="gray")


ax3 = sns.lineplot(x=df_assess.index, y="predicted", data=df_assess, color="blue")

plt.show()

### Plot the Absolute Error 

In [ ]:
plt.figure(figsize=(30, 10))


ax = sns.lineplot(x=df_assess.index, y="absolute_error", data=df_assess, color="red")
# ax.lines[0].set_linestyle("--")

# ax2 = sns.scatterplot(x=df_assess_day.index, y="absolute_error",
#             data=df_assess_day,
#             color="darkgray")

plt.show()

If we see a spike in the residual values at the approximate time of the tsunami-wave, we can use that information to explore ways of automating the detection of these disturbances. The approach to utilize for this classification is explored in the following sections. 

It should be noted that the ability to detect these disturbances must be thoroughly tested across multiple satellites and ground stations. 

### Plot the Smoothed Absolute Error

In [ ]:
# simple moving average 
df_assess["absolute_error_sma_3"] = df_assess["absolute_error"].rolling(3, min_periods=1).mean()
df_assess["absolute_error_sma_5"] = df_assess["absolute_error"].rolling(5, min_periods=1).mean()
df_assess["absolute_error_sma_10"] = df_assess["absolute_error"].rolling(10, min_periods=1).mean()

In [ ]:
plt.figure(figsize=(30, 10))


ax = sns.lineplot(x=df_assess.index, y="absolute_error", data=df_assess, color="gray")
ax = sns.lineplot(x=df_assess.index, y="absolute_error_sma_3", data=df_assess, color="red")
# ax.lines[0].set_linestyle("--")

# ax2 = sns.scatterplot(x=df_assess_day.index, y="absolute_error",
#             data=df_assess_day,
#             color="darkgray")

plt.show()

### Classify Time Periods As Anomalous

Model performance will generally always be impacted towards the start of a period (day) due to having less available data (data less than the specified batch size) for making predictions. This impacts model performance (confidence) at the start of any period, and thus confidence in the model will be less at early stages (until about an hour of data is collected). 

Once the event occurs, it seems to represent a large-enough deviation from the previous day(s) (used for model training) to result in elevated absolute errors for each prediction by the model after the change begins to occur in the the TEC variation. We can use this emperical finding to create a method for detection. 

We will not be able to do any better than an on-time detection of the event (meaning, there will be some lag between the start of the event and the detection of the event). We can score the approach and generate metrics (described further below) based on some specified threshold of acceptability of detection, e.g. 5 minutes (or something else). 

In [ ]:
# definition of start times for tsunami-wave induced pertubations by satellite
# start time defined by second in day 
sod_annotations = {
    "G04": 31400,
    "G07": 31160,
    "G08": 31900,
    "G10": 29900,
    "G20": 31150
}

Let's create an improved version of an earlier plot showing absolute errors that highlights the period of increased model confidence / performance at the start of the period and also has a vertical line indicating the start time of the pertubation in TEC variation. This plot will also show the time period in which we classify as anomalous. 

In [ ]:
sod_annotations

In [ ]:
plt.figure(figsize=(30, 10))


ax = sns.lineplot(x=df_assess["sod"], y="absolute_error", data=df_assess, color="lightgray")
ax = sns.lineplot(x=df_assess["sod"], y="absolute_error_sma_3", data=df_assess, color="black")
ax.axvline(sod_annotations[SAT], color="blue", linestyle="--")
plt.text(sod_annotations[SAT] + 80, np.max(df_assess["absolute_error"].values) / 2, "Start of Event", rotation=90, verticalalignment='center', color="blue")
ax.add_patch(
    patches.Rectangle(
        (0, 0), 
        df_assess.iloc[BATCH_SIZE]["sod"],
        np.max(df_assess["absolute_error"].values),
        color="gray",
        alpha=0.3
    )
)
ax.set_title('Absolute Error - Day of Earthquake - Satellite ' + SAT)

# ax2 = sns.lineplot(x=df_assess["sod"], y="absolute_error", data=df_assess, color="black")

plt.show()

Goal to develop approach of handling residual values for detection. 

In [ ]:
# mean of abs errors 
np.mean(df_assess["absolute_error"])

In [ ]:
# standard devation
np.std(df_assess["absolute_error"])

#### Distribution of Errors 

A lot of error handling approaches and work assume that the errors have Gaussian distributions. Let's visualize the distribution of values and do a formal test for normality. If we find that the distribution of errors is generally normal, we could use approaches that are suitable for that type of data distribution. If not, we should look at other approaches for handling the model errors. 

In [ ]:
# distribution of the errors
plt.figure(figsize=(10, 10))
sns.distplot(df_assess["absolute_error"].values, kde=True, rug=True);
plt.show()

Let's plot a QQ plot. This will provide a visual indication as to the normality of the data. If the points closely follow the diagonal line, then they closely fit the pattern we would expect from a Gaussian distribution. 

In [ ]:
qq = stats.probplot(df_assess["absolute_error"].values, dist='lognorm', sparams=(1))
x = np.array([qq[0][0][0], qq[0][0][-1]])

fig = go.Figure()
fig.add_scatter(x=qq[0][0], y=qq[0][1], mode='markers')
fig.add_scatter(x=x, y=qq[1][1] + qq[1][0]*x, mode='lines')
fig.layout.update(showlegend=False)
fig.show()

#### Normality Tests

In [ ]:
# Shapiro normality test
stat, p = stats.shapiro(df_assess["absolute_error"].values)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# D’Agostino’s K^2  normality test
stat, p = stats.normaltest(df_assess["absolute_error"].values)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# Anderson-Darling normality test
result = stats.anderson(df_assess["absolute_error"].values)
print('Statistic: %.3f' % result.statistic)
p = 0
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
    else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

#### Using a Time-based Distancing Approach 

For each observation, get the distances from each of the previous observations from _T_ steps prior (so the difference / distance in absolute errors). Then, log transform these distances to prioritize local anomalies (inspired from approach by Ian Colwell for MSL anomaly detection). 

In [ ]:
# number of previous time steps T to consider
DIST_WINDOW = 10

In [ ]:
def calc_dist_measure(dataframe: pd.DataFrame, window_size: int = 10, show_dist_plot: bool = False) -> pd.DataFrame:

    """
    Calculates the distance 
    """
    
    # create a dataframe for the errors 
    df_error = dataframe[["sod", "absolute_error_sma_3"]].iloc[window_size:].reset_index().sort_values(by="sod")
    
    # create a dataframe to store distances between absolute errors
    # more specifically, to store the mean distance from an observation 
    # to its previous N neighbors 
    df_error_dist = pd.DataFrame(list(), index=df_error["sod"], columns=df_error["sod"])
    for idx, row in tqdm(df_error.iterrows(), total=df_error.shape[0]):
    
        # get the current absolute error 
        abs_error = row["absolute_error_sma_3"]

        # get the absolute error values for the previous N values
        prev = dataframe.iloc[idx:idx+window_size] # note indexing to account for window size 
        
        # get the distances 
        for idx2, row2 in prev.iterrows():
            
            # TODO: could apply a weighted average here 
            
            dist = spatial.distance.euclidean(abs_error, row2["absolute_error_sma_3"])    
            # note log transformed values here :
            df_error_dist.loc[row["sod"], row2["sod"]] = np.log(dist + 1.)
#             df_error_dist.loc[row["sod"], row2["sod"]] = dist
            df_error_dist.loc[row2["sod"], row["sod"]] = np.log(dist + 1.) 
#             df_error_dist.loc[row2["sod"], row["sod"]] = dist 
            
    # optionally show a plot 
    if show_dist_plot is True:
        plt.figure(figsize=(15, 10))
        ax = sns.heatmap(df_error_dist.sort_index().sort_index(axis=1).fillna(0.))
        plt.show()
        
    # use that distance matrix to get the min/max/mean distance in the last window  
    # we only want to use observations with a full window size 
    counts = df_error_dist.count().reset_index().rename(columns={0: "counts"})
    full_sods = counts[counts["counts"] >= window_size]["sod"]
    
#     df_error_dist_flat = df_error_dist.loc[full_sods].mean(axis=1)
    df_error_dist_flat = df_error_dist.loc[full_sods].min(axis=1)
#     df_error_dist_flat = df_error_dist.loc[full_sods].max(axis=1)
    df_error_dist_flat = df_error_dist_flat.reset_index().rename(columns={0: "dist_measure"})

    return df_error_dist_flat

In [ ]:
df_test_dist_calc = calc_dist_measure(
    df_test,
    window_size=DIST_WINDOW,
    show_dist_plot=True
)

In [ ]:
df_test_dist_calc["dist_measure"].shape

In [ ]:
df_test.shape

In [ ]:
def get_val(dataframe, row):
    try:
        return dataframe[dataframe["sod"] == row]["dist_measure"].values[0]
    except:
        return 0.

In [ ]:
df_test["dist_measure"] = [0.] * df_test.shape[0]
df_test["dist_measure"] = df_test["sod"].apply(
    lambda x: get_val(df_test_dist_calc, x)
)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df_test["dist_measure_scaled"] = scaler.fit_transform([df_test["dist_measure"].values]).T

# df_test["dist_measure"]

In [ ]:
sns.distplot(df_test["dist_measure"].values, kde=True, rug=True);

In [ ]:
df_test["dist_measure"].describe()

In [ ]:
m_test = loop.LocalOutlierProbability(
    df_test.iloc[BATCH_SIZE:]["absolute_error"].values, 
    extent=3, 
    n_neighbors=30
).fit()
m_test_scores = m_test.local_outlier_probabilities
df_test["loop_scores"] = list([0.] * BATCH_SIZE) + list(m_test_scores)

In [ ]:
VMAX = 0.01

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(40, 10))
grid = plt.GridSpec(4, 8, wspace=0.4, hspace=0.3)

plt.subplot(grid[1:3, 0:8])

# ax1 = plt.figure(figsize=(30, 10))
ax1 = sns.lineplot(x=df_test["sod"], y="absolute_error", data=df_test, color="black")
ax1.margins(x=0)

# add a vertical line to indicate when a full window of distance values is available 
ax1.axvline(df_test.iloc[DIST_WINDOW]["sod"], color="purple", linestyle="--")
ax1.text(df_test.iloc[DIST_WINDOW]["sod"] + 80, np.max(df_assess["absolute_error"].values) / 2, "Window Buffer", rotation=90, verticalalignment='center', color="purple")

plt.subplot(grid[0, 0:8])
ax2 = sns.heatmap(
    [df_test["dist_measure"].values],
    cmap="bwr",
    cbar=False,
    xticklabels=False,
    yticklabels=False,
    vmin=0, 
    vmax=VMAX
)


plt.show()

In [ ]:
df_assess_dist_calc = calc_dist_measure(
    df_assess,
    window_size=DIST_WINDOW,
    show_dist_plot=False
)

In [ ]:
df_assess["dist_measure"] = [0.] * df_assess.shape[0]
df_assess["dist_measure"] = df_assess["sod"].apply(
    lambda x: get_val(df_assess_dist_calc, x)
)

In [ ]:
sns.distplot(df_assess["dist_measure"].values, kde=True, rug=True);

In [ ]:
df_assess["dist_measure"].describe()

In [ ]:
df_assess.shape

In [ ]:
m_assess = loop.LocalOutlierProbability(
    df_assess.iloc[BATCH_SIZE:]["absolute_error"].values, 
    extent=3, 
    n_neighbors=30
).fit()
m_assess_scores = m_assess.local_outlier_probabilities
df_assess["loop_scores"] = list([0.] * BATCH_SIZE) + list(m_assess_scores)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(40, 10))
grid = plt.GridSpec(4, 8, wspace=0.4, hspace=0.3)

plt.subplot(grid[1:3, 0:8])

# ax1 = plt.figure(figsize=(30, 10))
ax1 = sns.lineplot(x=df_assess["sod"], y="absolute_error", data=df_assess, color="black")
ax1.margins(x=0)

# add a vertical line to indicate the event 
ax1.axvline(sod_annotations[SAT], color="blue", linestyle="--")
ax1.text(sod_annotations[SAT] + 80, np.max(df_assess["absolute_error"].values) / 2, "Start of Event", rotation=90, verticalalignment='center', color="blue")

# add a vertical line to indicate when a full window of distance values is available 
ax1.axvline(df_assess.iloc[DIST_WINDOW]["sod"], color="purple", linestyle="--")
ax1.text(df_assess.iloc[DIST_WINDOW]["sod"] + 80, np.max(df_assess["absolute_error"].values) / 2, "Window Buffer", rotation=90, verticalalignment='center', color="purple")


plt.subplot(grid[0, 0:8])
ax2 = sns.heatmap(
    [df_assess["dist_measure"].values],
    cmap="bwr",
    cbar=False,
    xticklabels=False,
    yticklabels=False,
    vmin=0, 
    vmax=VMAX
)

plt.show()

## Assess Model and Approach to Classification

We need to report the following metrics: 

- **Root Mean Square Error**: measure of the differences between values (sample or population values) predicted by a model or an estimator and the values observed
- **Accuracy**: the number of correct classifications over the number of observations
- **Recall**: fraction of true events that were detected
- **Precision**: fraction of detections reported  by the model that are correct
- **F-Score**: the harmonic mean of the precision and recall, `2pr / (p + r)`
- **Coverage**: fraction of examples for which the system is able to produce a confident classification